In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context
from utils.utils_tx import deploy, endpoint_call, upgrade_call
from multiversx_sdk_core import CodeMetadata

context = Context()

In [ ]:
from utils.utils_chain import WrapperAddress, Account

account = Account(pem_file="~/Documents/sh0.pem")
account.sync_nonce(context.network_provider.proxy)
admin = Account(pem_file="~/Documents/sh02.pem")
admin.sync_nonce(context.network_provider.proxy)
user = Account(pem_file="~/Documents/sh1.pem")
user.sync_nonce(context.network_provider.proxy)

In [ ]:
deployer_address = 'erd1qqqqqqqqqqqqqpgq437hh07ahfncl7zhcfrsy6l6lttxgxnypauqd0d5gm'
deployer_contract = WrapperAddress(deployer_address)

deploy minter

In [ ]:
bytecode = Path.home() / "projects" / "mx-nft-collection-minter-sc" / "output-docker" / "nft-minter" / "nft-minter.wasm"
metadata = CodeMetadata(upgradeable=True, payable_by_contract=True, readable=True)
hash, template_address = deploy("minter", context.network_provider.proxy, 100000000, account, bytecode, metadata, [
    account.address,
    account.address,
    10
])

deploy deployer

In [ ]:
bytecode = Path.home() / "projects" / "mx-nft-collection-minter-sc" / "output-docker" / "nft-minter-deployer" / "nft-minter-deployer.wasm"
metadata = CodeMetadata(upgradeable=True, payable_by_contract=True, readable=True)
hash, deployer_address = deploy("deployer", context.network_provider.proxy, 100000000, account, bytecode, metadata, [
    WrapperAddress("erd1qqqqqqqqqqqqqpgqtemckxhuuuw7efyg6q4604yevfv4xd4zpauq9u4kyr"),
])
deployer_contract = WrapperAddress(deployer_address)

creation enable

In [ ]:
endpoint_call(context.network_provider.proxy, 30000000, account, 
              WrapperAddress(deployer_address), "setNftMinterCreationEnabled", [1])

pause minter

In [ ]:
# minter_contract = WrapperAddress('erd1qqqqqqqqqqqqqpgq8tce96eswecvkp48ej95w32kdge4my76pauq7fwkp7')
endpoint_call(context.network_provider.proxy, 30000000, account, deployer_contract, "pauseNftMinter", [
                  minter_contract])

resume minter

In [ ]:
# minter_contract = WrapperAddress('erd1qqqqqqqqqqqqqpgq8tce96eswecvkp48ej95w32kdge4my76pauq7fwkp7')
endpoint_call(context.network_provider.proxy, 30000000, account, deployer_contract, "resumeNftMinter", [
                  minter_contract])

add admin to minter

In [ ]:
# minter_contract = WrapperAddress('erd1qqqqqqqqqqqqqpgq8tce96eswecvkp48ej95w32kdge4my76pauq7fwkp7')
endpoint_call(context.network_provider.proxy, 30000000, account, deployer_contract, "addAdminToNftMinterContract", [
                  admin.address, minter_contract])

remove admin to minter

In [ ]:
# minter_contract = WrapperAddress('erd1qqqqqqqqqqqqqpgq8tce96eswecvkp48ej95w32kdge4my76pauq7fwkp7')
endpoint_call(context.network_provider.proxy, 30000000, account, deployer_contract, "removeAdminToNftMinterContract", [
                  user.address, minter_contract])

deploy minter via deployer

In [ ]:
hash = endpoint_call(context.network_provider.proxy, 30000000, admin, 
              WrapperAddress(deployer_address), "createNftMinter", [
                  admin.address,
                  admin.address,
                  10
              ])

In [ ]:
# hash = '9474be478889788cd2573e3f3008907e7902cbd1d47678fbb939c419c8255ea9'
tx_data = context.network_provider.proxy.get_transaction(hash)
if tx_data.status.is_failed():
    print("Transaction failed")
    exit(1)
event = tx_data.logs.find_first_or_none_event("SCDeploy")
topic = event.topics[0]
minter_contract = WrapperAddress.from_hex(topic.hex(), "erd")
print("Deployed minter:", minter_contract.bech32())

set royalties address in minter

In [ ]:
hash = endpoint_call(context.network_provider.proxy, 30000000, admin, minter_contract, "setRoyaltiesClaimAddress", [
                  admin.address,
              ])